In [1]:
from tqdm import tqdm 
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import xdeepfm
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import DataLoader, TensorDataset
from xdeepfm import Field

df = pd.read_csv("./data/ml-100k/u.data", header=None, delim_whitespace=True, encoding='utf-8')
df.columns = ['user_id', 'item_id', 'rating', 'timestamp']

num_users = np.max(df['user_id']).item() + 1
num_items = np.max(df['item_id']).item() + 1
fields = [xdeepfm.Field('user_id', num_users), xdeepfm.Field('item_id', num_items)]

In [2]:
%load_ext autoreload
%autoreload 2

In [7]:
df = pd.read_csv("./data/ml-100k/u.data", header=None, delim_whitespace=True, encoding='utf-8')
df.columns = ['user_id', 'item_id', 'rating', 'timestamp']

X = df[['user_id', 'item_id']].to_numpy()
y = df['rating'].to_numpy()
y = (y >= 4).astype(int)

batch_size = 2048

num_users = np.max(df['user_id']).item() + 1
num_items = np.max(df['item_id']).item() + 1
fields = [xdeepfm.Field('user_id', num_users), xdeepfm.Field('item_id', num_items)]

train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.5)
train_dl = DataLoader(
    TensorDataset(
        torch.tensor(train_X),
        torch.tensor(train_y).float(),
    ),
    batch_size=batch_size,
)
test_dl = DataLoader(
    TensorDataset(
        torch.tensor(test_X),
        torch.tensor(test_y).float(),
    ),
    batch_size=batch_size,
)

def fit(net, loss_fn, epochs=10, lr=0.001):
    optim = torch.optim.Adam(net.parameters(), lr)
    net.train()
    net.cuda()
    J = []  
    for e in tqdm(range(epochs)):
        for batch_x, batch_y in tqdm(train_dl):
            batch_x, batch_y = batch_x.cuda(), batch_y.cuda()
            optim.zero_grad()
            batch_y = batch_y.unsqueeze(-1)
            out = net(batch_x)
            loss = loss_fn(out, batch_y)
            loss.backward()
            optim.step()
            J.append(loss)
            break
        print(f"Epoch {e + 1}/{epochs}: Loss {loss:.2f}")
    return J

In [12]:
config = xdeepfm.NetConfig(fields=fields, m=2, dnn=xdeepfm.DNNConfig(), cin=xdeepfm.CINConfig())
net = xdeepfm.xDeepFM(config=config)
J = fit(net, torch.nn.BCELoss())

 10%|█         | 1/10 [00:05<00:48,  5.44s/it]

tensor(0.6954, grad_fn=<BinaryCrossEntropyBackward0>)
Epoch 1/10: Loss 0.70


 20%|██        | 2/10 [00:11<00:45,  5.70s/it]

tensor(0.6653, grad_fn=<BinaryCrossEntropyBackward0>)
Epoch 2/10: Loss 0.67


 30%|███       | 3/10 [00:16<00:37,  5.43s/it]

...

 90%|█████████ | 9/10 [00:44<00:04,  4.82s/it]

tensor(0.4900, grad_fn=<BinaryCrossEntropyBackward0>)
Epoch 9/10: Loss 0.49


100%|██████████| 10/10 [00:49<00:00,  4.96s/it]

tensor(0.4625, grad_fn=<BinaryCrossEntropyBackward0>)
Epoch 10/10: Loss 0.46


[tensor(0.6954, grad_fn=<BinaryCrossEntropyBackward0>),
 tensor(0.6653, grad_fn=<BinaryCrossEntropyBackward0>),
 tensor(0.6385, grad_fn=<BinaryCrossEntropyBackward0>),
 tensor(0.6131, grad_fn=<BinaryCrossEntropyBackward0>),
 tensor(0.5886, grad_fn=<BinaryCrossEntropyBackward0>),
 tensor(0.5652, grad_fn=<BinaryCrossEntropyBackward0>),
 tensor(0.5415, grad_fn=<BinaryCrossEntropyBackward0>),
 tensor(0.5164, grad_fn=<BinaryCrossEntropyBackward0>),
 tensor(0.4900, grad_fn=<BinaryCrossEntropyBackward0>),
 tensor(0.4625, grad_fn=<BinaryCrossEntropyBackward0>)]